In [1]:
#Fake News Classifier

Dataset: https://www.kaggle.com/c/fake-news/data#

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('../DataSets/train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull()

,id,title,author,text,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
20795,False,False,False,False,False
20796,False,False,False,False,False
20797,False,False,False,False,False
20798,False,False,False,False,False


In [8]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
df=df.dropna()

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
X=df.drop('label',axis=1)

In [14]:
y=df['label']

In [15]:
X.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [32]:
#vocabulary size
voc_size=5000

In [20]:
messages=X.copy()

In [21]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [22]:
# messages.reset_index(inplace=True)

In [23]:
messages.sample(10)

,index,id,title,author,text
6428,7328,7328,Trump: Polling Machines Are Flipping Votes For...,Edmondo Burr,"Edmondo Burr in News , US // 2 Comments Dona..."
8476,9661,9661,"EXCLUSIVE: Cartel Smugglers, Migrants Growing ...",Brandon Darby,"TUCSON, Arizona — Sinaloa smugglers and m..."
2098,2408,2408,Israeli Embassy Official Plotted to ‘Take Down...,Steven Erlanger,LONDON — In a deeply embarrassing episode r...
10820,12293,12293,"Warning of ISIS Plots Against West, U.S. Plans...",Helene Cooper and Eric Schmitt,"WASHINGTON — The fight to retake Raqqa, the..."
3158,3632,3632,BREAKING: What Obama Just Did To A Veteran Wil...,Starkman,Barack Obama has been out campaigning on behal...
5436,6207,6207,"Captagon and Western-backed rebels: New drug, ...",Makia Freeman,"Captagon and Western-backed rebels: New drug, ..."
518,601,601,ICE Union Issues Final Warning to Voters,Bob Adelmann,"Email \nIn September the National ICE Council,..."
12685,14428,14428,Chicago Is Planning to Add Hundreds of Police ...,Monica Davey,"CHICAGO — This city, reeling from a rising ..."
15324,17434,17434,Five Illuminating Takeaways From Your Family M...,Ron Lieber,When Joe Olivier asked what his niece wanted a...
710,818,818,"“He Won Because The Elites WANT HIM There, The...",Contributing Author,\nThis article was written by Melissa Dykes an...


In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RITIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
#PreProcessing

In [29]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [30]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [33]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [34]:
onehot_repr

[[745, 3477, 2389, 714, 2794, 4029, 1585, 1827, 635, 4148],
 [3057, 1859, 749, 3402, 2723, 648, 2311],
 [93, 4372, 1186, 1816],
 [1785, 779, 4524, 2003, 2482, 2551],
 [1110, 2723, 595, 3180, 452, 4105, 2723, 408, 4659, 2611],
 [4563,
  1702,
  751,
  663,
  1688,
  882,
  3059,
  1708,
  1235,
  2486,
  4163,
  885,
  2222,
  4097,
  2311],
 [4941, 943, 1006, 1673, 1122, 320, 3504, 1823, 41, 1811, 1219],
 [2406, 970, 2422, 1030, 3687, 4857, 882, 3965, 41, 1811, 1219],
 [4732, 877, 923, 2292, 1554, 2579, 587, 2919, 882, 3785],
 [2588, 2172, 951, 1901, 4368, 2026, 171, 1600],
 [2835, 3067, 2592, 2527, 932, 2755, 2780, 2233, 2976, 3776, 703],
 [2003, 4503, 2794, 2579, 882, 3687],
 [622, 1390, 1753, 1092, 3940, 2117, 2597, 2384, 416],
 [1858, 1603, 4619, 3098, 1278, 299, 227, 41, 1811, 1219],
 [4655, 2777, 1981, 2994, 77, 41, 1811, 1219],
 [2409, 4475, 2009, 1965, 2211, 1314, 2301, 3638, 4843, 1621],
 [3360, 3683, 1859],
 [3196, 1980, 2806, 543, 882, 4732, 390, 2311],
 [2595, 2307, 749, 37

In [35]:
#word embedding representation

In [36]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
embedded_docs

array([[ 745, 3477, 2389, ...,    0,    0,    0],
       [3057, 1859,  749, ...,    0,    0,    0],
       [  93, 4372, 1186, ...,    0,    0,    0],
       ...,
       [ 841, 4906, 3694, ...,    0,    0,    0],
       [4715, 3687, 2272, ...,    0,    0,    0],
       [ 829,  311,  227, ...,    0,    0,    0]])

In [38]:
from tensorflow.keras.models import Sequential

In [39]:
#model creation
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [41]:
len(embedded_docs)

18285

In [42]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [43]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [45]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 24s 17ms/step - loss: 0.3067 - accuracy: 0.8540 - val_loss: 0.1919 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1357 - accuracy: 0.9479 - val_loss: 0.1998 - val_accuracy: 0.9195
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0927 - accuracy: 0.9662 - val_loss: 0.2213 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0672 - accuracy: 0.9764 - val_loss: 0.2606 - val_accuracy: 0.9082
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0478 - accuracy: 0.9854 - val_loss: 0.2934 - val_accuracy: 0.9130
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0331 - accuracy: 0.9896 - val_loss: 0.3525 - val_accuracy: 0.9128
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0268 - accuracy: 0.9922 - val_loss: 0.4023 - val_accuracy: 0.911

In [46]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [47]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [48]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test,y_pred)

array([[3123,  296],
       [ 231, 2385]], dtype=int64)

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9126760563380282

In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

